In [1]:
from elasticsearch import Elasticsearch
import json

import os
import re
import numpy as np

from tqdm import tqdm
from pprint import pprint
import time
from contextlib import contextmanager
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")

# 1. INDEX 설정 및 생성

In [2]:
# doc data read
with open("../../data/wikipedia_documents.json") as f:
    doc_data = json.load(f)

contexts = list(
            dict.fromkeys([v["text"] for v in doc_data.values()])
        )

In [3]:
print(doc_data['0'].items())

dict_items([('text', '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'), ('corpus_source', '위키피디아'), ('url', 'TODO'), ('domain', None), ('title', '나라 목록'), ('author', None), ('html', None), ('document_id', 0)])


In [4]:
INDEX_NAME = "wiki_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        #"title" : {
        #  "type" : "text",
        #  "analyzer": "korean",
        #  "search_analyzer": "korean"
        #}
      }

  }
}

In [5]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [6]:
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '98b1f63e63e9',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'KBJP5UXDQEmZ1yku1tyaIQ',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [7]:
#
# 인덱스 생성
# *인덱스가 이미 존재하면 삭제됨*
#

if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-7-a639291b115c>:6: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-7-a639291b115c>:8: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_index'}

# 2. 데이터 추가

In [8]:
#
# 데이터 인덱싱
# *인덱스 새로 생성했을때만 실행*
#

for doc_id, doc in enumerate(tqdm(contexts)):
    es.index(index=INDEX_NAME,  id=doc_id, body={'content':doc})
    #time.sleep(1)

  0%|          | 0/56737 [00:00<?, ?it/s]<ipython-input-8-0834f0f292ff>:7: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body={'content':doc})
100%|██████████| 56737/56737 [04:30<00:00, 209.70it/s]


In [9]:
# 인덱싱한 문서 확인

doc = es.get(index=INDEX_NAME, id=0)
pprint(doc)

{'_id': '0',
 '_index': 'wiki_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'content': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 '
                        '나열하고 있다.\n'
                        '\n'
                        '이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n'
                        '\n'
                        '# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 '
                        '승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n'
                        '# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 '
                        '승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n'
                        '\n'
                        '두 목록은 모두 가나다 순이다.\n'
                        '\n'
                        '일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 '
                        '매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 '
                        '대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" '
                        '문서에서 설명하고 있다.'},
 '_type': '_doc',
 '_version'

In [10]:
# 

#tv = es.termvectors(index=INDEX_NAME, id=0,body={"fields" : ["content"]})
#pprint(tv)

# 3. Elastic Search Test

In [11]:
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
    load_metric,
    Sequence,
    Value,
    Features,
    Dataset,
    DatasetDict,
)

# 테스트 해볼 데이터
datasets = load_from_disk('../../data/train_dataset')

In [12]:
datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [13]:
from pprint import pprint
val_question = datasets['validation']['question']
val_context = datasets['validation']['context']
pprint(val_question[2])
pprint(val_context[2])

'촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?'
('촐라 정부\\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권력을 지니고 있었다. 촐라의 왕은 만디르를 자주 '
 '건설하였는데, 촐라 제국의 만디르는 단순히 종교적 시설로서의 역할뿐만 아니라 경제 활동의 중심지 역할 또한 '
 '수행하였다.2003\\n\\n라자라자 촐라 이전에는 수도 근방은 촐라 황제가 통치하고 외곽 지역은 촐라 황제와 유력한 관계를 지녔던 '
 '벨리르(토후)에 의해 통치되는 봉건제 형식이었지만, 라자라자 1세 이후 촐라 제국의 행정 구역은 8개의 만달람(속주)으로 '
 '세분화되었는데,2008 만달람은 발라나두(군), 발라나두는 코람(현) 등으로 세분화되었고, 코람은 행정의 가장 작은 단위인 몇 개의 '
 '촌락으로 세분화되었으며, 촌락은 30여 개의 구로 나뉘어져 있었다.\\n\\n이들 행정 구역들의 지방관들은 초기에는 중앙정부에 의해 '
 '임명되었지만 후기에는 그 권한을 세습하였으며, 촌락에서는 행정·물 관리·사법 등 마을에 필요한 위원회를 마련한 후, 각 위원회 운영에 '
 '참여할 만한 식견을 갖춘 사람의 이름을 나뭇잎에 적어서 어린아이가 제비뽑기를 하는 방식으로 운영 위원을 선출하였으며 촐라 제국의 이 같은 '
 '자치 제도는 마을 고유의 행정과 문화가 유지되는데 중요한 역할을 했다.')


In [15]:
idx = 6
k = 5

query = val_question[idx]
ground_truth = val_context[idx]

print("[query]")
pprint(query)
print()
print("[ground truth]")
pprint(ground_truth)
print()

# query에 특수문자가 있으면 특수문자 앞에 "\" 문자 추가
patten = '''[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]'''
query = re.sub(patten, lambda m: f"\{m.group()}", query)

with timer("searching.."):
    res = es.search(index=INDEX_NAME, q=query, size=k)
print()

for i, hit in enumerate(res['hits']['hits']):
    print(f"[{i}] ID : {hit['_id']}, SCORE : {hit['_score']}")
    #pprint(hit['_source']['title'])
    pprint(hit['_source']['content'])
    print()

#res

[query]
'징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가?'

[ground truth]
('수를 놓을 때에 사용하는 기법을 문양의 형태에 따라 나누어 보면 선을 잇는 수, 면을 메우는 수, 어떤 무늬를 나타내는 수로 나눌 수 '
 '있다. 그 종류로는 아주 작은 점으로 표현되는 점수를 비롯해서 선을 표현하는 이음수, 실을 간격이 없이 고루 펴는 평수, 양쪽으로 갈라져 '
 '있는 잎을 수놓는 가름수, 수평으로 나란히 수놓는 푼사수, 서양 자수의 코칭 스티치와 같은 수법으로 노끈·굵은 실·금사·은사 같은 다른 '
 '가는 실로 징그어 주는 기법인 징금수, 색의 자연스러운 변화를 표현하는 데 이용되는 자련수, 돗자리의 겉모양을 표현하는 기법인 자릿수, '
 '겉수의 입체감을 나타내기 위해 양감(量感)을 표현하는 속수, 일정하게 간격을 띄우고 수평으로 수놓는 기법인 관수, 양지관수로 밑수를 '
 '수놓은 다음 밑수의 색과 유사한 색으로 실과 실사이를 어슷하게 건너주는 엇겨놓기수, 새의 깃털을 표현하는 새털수, 결의 방향을 규칙적으로 '
 '바꾸어 가며 무늬를 만드는 무늬목수, 삼나무 잎모양처럼 엮어나간 연속무늬의 기법인 삼잎수, 소나무 잎을 수놓을 때 쓰는 솔잎수, '
 '별모양처럼 표현하는 별무늬수, 금·은사를 사용하여 평사수로 밑수한 위에 징금수하여 기하학적인 문양을 만드는 칠보수 등이 있다.')

[searching..] done in 0.163 s

[0] ID : 16171, SCORE : 57.05222
('수를 놓을 때에 사용하는 기법을 문양의 형태에 따라 나누어 보면 선을 잇는 수, 면을 메우는 수, 어떤 무늬를 나타내는 수로 나눌 수 '
 '있다. 그 종류로는 아주 작은 점으로 표현되는 점수를 비롯해서 선을 표현하는 이음수, 실을 간격이 없이 고루 펴는 평수, 양쪽으로 갈라져 '
 '있는 잎을 수놓는 가름수, 수평으로 나란히 수놓는 푼사수, 서양 자수의 코칭 스티치와 같은 수법으로 노끈·굵은 실·금사·은사 같은 다른 '
 '가는 실

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
